In [ ]:
import sys
sys.path.append('../../bluesky')  # Adjust the path as necessary
import os
sys.path.append(os.path.abspath('../'))



from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from prompts.prompts import conflict_prompt
from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from bluesky.network.client import Client
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from dotenv import load_dotenv, find_dotenv
import time
from contextlib import contextmanager
from io import StringIO
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents.format_scratchpad.openai_tools import (

    format_to_openai_tool_messages,

)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage
import bluesky
import streamlit as st
from langchain_community.llms import Ollama
from langchain.agents import tool
from langchain.agents import tool, initialize_agent, AgentType, Tool
import os




# Now you can import bluesky modules



os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"
load_dotenv(find_dotenv())

In [ ]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-large"
)
vectordb_path = 'C:/Users/justa/OneDrive/Desktop/Developer/LLM-Enhanced-ATM/llm/skills-library/vectordb'
chroma_client = chromadb.PersistentClient(path=vectordb_path)
# Get a collection object from an existing collection, by name. If it doesn't exist, create it.
collection = chroma_client.get_or_create_collection(
    name="test2", embedding_function=openai_ef, metadata={"hnsw:space": "cosine"})

#######################################################################################


# capture output information from the bluesky client and return it as a string
@contextmanager
def capture_stdout():
    new_stdout = StringIO()
    old_stdout = sys.stdout
    sys.stdout = new_stdout
    try:
        yield new_stdout
    finally:
        sys.stdout = old_stdout


def update_until_complete(client):
    complete_output = ""
    empty_output_count = 0  # Track consecutive empty outputs

    while True:
        with capture_stdout() as captured:
            client.update()
        new_output = captured.getvalue()

        # Check if the current output is empty
        if not new_output.strip():
            empty_output_count += 1  # Increment counter for empty outputs
        else:
            empty_output_count = 0  # Reset counter if output is not empty
            complete_output += new_output  # Add non-empty output to complete output

        # If there are two consecutive empty outputs, break the loop
        if empty_output_count >= 2:
            break

    # It's assumed you want to keep the last update outside the loop
    client.update()

    return complete_output

#######################################################################################


# connect to client
client = Client()
client.connect("127.0.0.1", 11000, 11001)
client.update()
client.update()

In [ ]:
from typing import Annotated, List, Tuple, Union

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool


# get all aircraft info
# get conflict information

@tool
def GetAllAircraftInfo(command: str = 'GETACIDS'):
    """Get each aircraft information at current time: position, heading (deg), track (deg), altitude, V/S (vertical speed), calibrated, true and ground speed and mach number. Input is 'GETACIDS'.
    
    Parameters:
    - command: str (default 'GETACIDS')
    
    Example usage:
    - GetAllAircraftInfo('GETACIDS')
    
    Returns:
    - str: all aircraft information
    """
    command = command.replace('"', '').replace("'", "")
    command = command.split('\n')[0]
    print(f'LLM input:{command}')

    client.send_event(b'STACK', command)
    time.sleep(1)
    sim_output = update_until_complete(client)
    return sim_output


@tool
def GetConflictInfo(commad: str = 'SHOWTCPA'):
    """Use this tool to identify and get vital information on aircraft pairs in conflict. It gives you Time to Closest Point of Approach (TCPA), Quadrantal Direction (QDR), separation distance, Closest Point of Approach distance (DCPA), and Time of Loss of Separation (tLOS).
    
    Parameters:
    - command: str (default 'SHOWTCPA')
    
    Example usage:
    - GetConflictInfo('SHOWTCPA')
    
    Returns:
    - str: conflict information between aircraft pairs
    """
    client.send_event(b'STACK', 'SHOWTCPA')
    time.sleep(1)
    sim_output = update_until_complete(client)
    return sim_output


@tool
def ContinueMonitoring(duration: str = '5'):
    """Monitor for conflicts between aircraft pairs for a specified time. 
    Parameters:
    - duration (str): The time in seconds to monitor for conflicts. Default is 5 seconds.
    
    Example usage:
    - ContinueMonitoring('5')
    
    Returns:
    - str: The conflict information between aircraft pairs throughout the monitoring period.
    """
    for i in range(int(duration)):
        client.send_event(b'STACK', 'SHOWTCPA')
        time.sleep(1)
        sim_output += str(i) + ' sec: \n' + \
            update_until_complete(client) + '\n'
    return sim_output


@tool
def SendCommand(command: str):
    """
    Sends a command with optional arguments to the simulator and returns the output. 
    You can only send 1 command at a time.
    
    Parameters:
    - command (str): The command to send to the simulator. Can only be a single command, with no AND or OR operators.
    
    Example usage:
    - SendCommand('COMMAND_NAME ARG1 ARG2 ARG3 ...) # this command requires arguments
    - SendCommand('COMMAND_NAME') # this command does not require arguments
    
    Returns:
    str: The output from the simulator.
    """
    # Convert the command and its arguments into a string to be sent
    # command_with_args = ' '.join([command] + [str(arg) for arg in args])
    # Send the command to the simulator
    # client.update()  # Uncomment this if you need to update the client state before sending the command
    print(command)
    # replace " or ' in the command string with nothing
    command = command.replace('"', '').replace("'", "")
    command = command.split('\n')[0]
    client.send_event(b'STACK', command)
    # wait 1 second
    time.sleep(1)
    # Wait for and retrieve the output from the simulator
    sim_output = update_until_complete(client)
    if sim_output == '':
        return 'Command executed successfully.'
    if 'Unknown command' in sim_output:
        return sim_output + '\n' + 'Please use a tool QueryDatabase to search for the correct command.'
    return sim_output


@tool
def QueryDatabase(input: List[str]):
    """Use the tool to search for possible commands that can be sent to a simulator. The tool will return the top 5 commands that are similar to the input query. You can ask multiple questions at once.
    Example usage:
    - QueryDatabase(['Change heading of the aircraft', 'add waypoint to the aircraft'])
    Output:
    [[doc 1 for query 1, doc 2 for query 1, doc 3 for query 1], [doc 1 for query 2, doc 2 for query 2, ...], ...]
    """

    query_results = collection.query(
        query_texts=input,
        n_results=5
    )
    
    return query_results['documents']

In [ ]:
from langchain import hub

with open('../prompts/basecmds.txt', 'r') as file:
    base_cmds = file.read()

openai_function_prompt = hub.pull("hwchase17/openai-functions-agent")
openai_function_prompt.messages[0].prompt.template = f"""

These are the Base commands in simulation: {base_cmds}

Your Task is:
1.Analyze the Input Command: Understand and interpret the high-level command provided by the user.
2. Identify Key Actions: Break down the command into its fundamental components or steps necessary to achieve the command.
3. Associate Commands: For each identified step, determine the relevant commands from the base command set that will execute or facilitate the step.
4. Formulate Step Descriptions: Write a concise description for each step, explaining its purpose and how it contributes to the overall task.
5. Compile Command Syntax: List the exact commands for each step

Input: Command
Output format:
Step-by-Step Plan for Command:
- Step 1: Description of what needs to be accomplished in this step
  Commands: [Command1, Command2, ...]
- Step 2: Description of what needs to be accomplished in this step
  Commands: [Command1, Command2, ...]
- Step 3: [Description of what needs to be accomplished in this step]
  Commands: [Command1, Command2, ...]
- ...


extra Commands: GetAllAircraftInfo - will output all aircraft information
                GetConflictInfo - will output conflict information
                ContinueMonitoring - will output conflict information for a specified time

"""
#openai_function_prompt.pretty_print()

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent, create_json_chat_agent, create_xml_agent, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional

prompt1 = ChatPromptTemplate.from_messages([
    ("system", openai_function_prompt.messages[0].prompt.template),
    ("user", "{input}")
])

prompt2 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)
# model='gpt-4-turbo-2024-04-09'
llm = ChatOpenAI()



class Plan(BaseModel):
    """Step by step plan and commands for each plan"""

    # ^ Doc-string for the plan.
    # This doc-string is sent to the LLM as the description of the schema Plan,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    step: Optional[list] = Field(
        default=None, description="step number and description of the step")
    commands_lst: Optional[List[list]] = Field(
        default=None, description="the names of the commands for the step"
    )


chain1 = prompt1 | llm | StrOutputParser() | llm.with_structured_output(schema=Plan)

# chain2 = {"text": chain1} | prompt2 |  llm.with_structured_output(schema=Plan)
# chain2.invoke({'input': 'Resolve all conflicts in the airspace'})

In [ ]:
chain1.invoke({'input': 'Resolve all conflicts in the airspace'})	

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent, OpenAIFunctionsAgent, create_react_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["Verificator", "Monitor", "Controller"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH. Verificator - it verifies if there are still conflicts in the airspace after controller tried to resolve it. Monitor - it monitors the airspace and provides all aircraft detailed information. Controller - it resolves conflicts in the airspace. by sending commands to the simulator."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}. Verificator must always be before FINISH.",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

# gpt-3.5-turbo-0125
# gpt-4-turbo-2024-04-09
llm = ChatOpenAI(model="gpt-4-turbo-2024-04-09")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [ ]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


# tools = [GetAllAircraftInfo, GetConflictInfo, ContinueMonitoring, SendCommand, QueryDatabase]

monitor_prompt = "You are part of the conflict detection and resolution team. You are responsible for monitoring the airspace for conflict and aircraft information."

verificator_prompt = "You are part of the conflict detection and resolution team. You are responsible for verifying that there are no conflicts in the airspace or that all conflicts have been resolved. You always must use Tool GetConflictInfo which will tell you exactly either conflict information if there are conflicts or 'No conflicts' if there are no conflicts."

Controller_prompt = "You are part of the conflict detection and resolution team. You are responsible for sending commands to the simulator and searching for possible commands, if you don't know what command to send. Tool SendCommand will send a command to the simulator and return the output. Tool QueryDatabase will search for possible commands that can be sent to the simulator."

monitor_agent = create_agent(
    llm, [GetAllAircraftInfo, GetConflictInfo, ContinueMonitoring], monitor_prompt)
monitor_node = functools.partial(
    agent_node, agent=monitor_agent, name="Monitor")


verification_agent = create_agent(
    llm,
    [GetAllAircraftInfo, GetConflictInfo, ContinueMonitoring],
    "You are responsable for verifying if the user request has been succesfully executed.",
)
verification_node = functools.partial(
    agent_node, agent=verification_agent, name="Verificator")

controller_agent = create_agent(
    llm,
    [SendCommand, QueryDatabase],
    "You are responsable for sending commands to the simulator and searching for possible commands.",
)
controller_node = functools.partial(
    agent_node, agent=controller_agent, name="Controller")


workflow = StateGraph(AgentState)
workflow.add_node("Monitor", monitor_node)
workflow.add_node("Verificator", verification_node)
workflow.add_node("Controller", controller_node)
workflow.add_node("supervisor", supervisor_chain)

In [ ]:

for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges(
    "supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [ ]:
client.send_event(b'STACK', 'IC simple/conflicts/2ac/case3.scn')
update_until_complete(client)
update_until_complete(client)

In [ ]:
update_until_complete(client)

In [ ]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Please resolve any current conflict in the airspace",)
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")